## Manifold learning in Power System Transient Stability Assessment

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.manifold import Isomap, TSNE, MDS
from sklearn.manifold import SpectralEmbedding as SE
from sklearn.manifold import LocallyLinearEmbedding as LLE
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics.pairwise import paired_distances

In [ ]:
# Using experimental HalvingRandomSearchCV for hyperparameters optimization.
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV

In [ ]:
from scipy import stats

In [ ]:
from annealing import simulated_annealing

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
# Figure aesthetics
sns.set(context='paper', style='white', font_scale=1.1)
sns.set_style('ticks', {'xtick.direction':'in', 'ytick.direction':'in'})

#### Power System Transient Stability Analysis Data (IEEE Benchmark Test Case)

In [ ]:
data = pd.read_csv('GridDictionary2.csv')
data.head()

In [ ]:
# print(data.columns.values)

In [ ]:
# Percentage of "ones" in the "Stability" column.
print('There is {:.1f}% of unstable cases in the dataset!'
      .format(data['Stability'].sum()/float(len(data['Stability']))*100.))

In [ ]:
no_features = len(data.columns) - 1
X_data = data.iloc[:, 0:no_features]  # features
print('X_data', X_data.shape)
y_data = data['Stability']
print('y_data', y_data.shape)

#### Stratify shuffle split

In [ ]:
# Split dataset into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, train_size=0.8, stratify=y_data, shuffle=True)

In [ ]:
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

In [ ]:
print('Unstable cases in training dataset: {:.1f}%:'
      .format(np.sum(y_train)/float(len(y_train))*100.))
print('Unstable cases in testing dataset {:.1f}%:'
      .format(np.sum(y_test)/float(len(y_test))*100.))

In [ ]:
# Stable cases index values.
idx_stable = y_test==0

#### Scoring models using cross-validated metrics

In [ ]:
def score_default(X, y):
    """ Scoring default SVC model. """
    # Score with default hyperparameters.
    scores = cross_val_score(svm.SVC(kernel='rbf', class_weight='balanced'), 
                             X, y, cv=3, scoring='f1')
    print('Score using 3-fold CV: {:g} +/- {:g}'
          .format(np.mean(scores), np.std(scores)))
    return np.mean(scores), np.std(scores)

In [ ]:
def score_optimized(X, y, C, gamma):
    """ Scoring optimized SVC model. """
    # Score with the optimized hyperparameters.
    scores = cross_val_score(svm.SVC(C=C, gamma=gamma, kernel='rbf', 
                                     class_weight='balanced'), 
                             X, y, cv=3, scoring='f1', n_jobs=-1)
    print('Score using 3-fold CV: {:g} +/- {:g}'
          .format(np.mean(scores), np.std(scores)))
    return np.mean(scores), np.std(scores)

In [ ]:
def plot_projection(X, idx):
    fig, ax = plt.subplots(figsize=(4,4))
    ax.scatter(X[idx,0], X[idx,1], 
            s=20, c='green', marker='o', edgecolors='k', alpha=0.5, label='Stable')
    ax.scatter(X[~idx,0], X[~idx,1], 
            s=20, c='red', marker='o', edgecolors='k', alpha=0.5, label='Unstable')
    ax.legend(loc='best')
    ax.set_xlabel('First component')
    ax.set_ylabel('Second component')
    ax.grid()
    fig.tight_layout()
    plt.show()

#### StandardScaler

In [ ]:
# Standardize the input data.
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Following dimmensionality reduction methods are examined:

* **PCA** (principal components analysis)
* **kPCA** (kernelized principal components analysis)
* **tSVD** (truncated singular value decomposition)
* **iMAP** (isomap embedding)
* **t-SNE** (T-distributed stochastic neighbor embedding)
* **LLE** (locally linear embedding)
* **LLE:LTSA** (locally linear embedding with local tangent space alignment algorithm)
* **LLE:H** (locally linear embedding with Hessian eigenmap method)
* **SE** (spectral embedding)
* **MDS** (multi-dimensional scaling)

Some of these have their own hyperparameters (e.g. KernelPCA) which can be optimized together with the hyperparameters of the SVC estimator. This will be shown for the KernelPCA method.

### Hyperparameter optimization with simulated annealing

Simulated annealing is used for optimizing hyperparameters of the SVC estimator only.

In [ ]:
def svc_cv(C, gamma, X_data, y_data):
    """ 
    Support Vector Machine Classifier cross-validation.
    
    This function will instantiate a SVC classifier with a 
    RBF kernel and hyper-parameters "C" and "gamma". Combined
    with data and targets it will be used to perform cross-
    validation. The goal is to find combinations of "C" and
    "gamma" that maximizes the `f1` scoring metric.
    
    Parameters
    ----------
    C: float
        Regularization parameter (penalty is a squared l2). 
    gamma: float
        Kernel coefficient.
    X_data: np.array
        Matrix (2d array) of features.
    y_data: np.array
        Vector (1d array) of targets.
    
    Returns
    -------
    cval: float
        Mean value of the score from the cross-validation.
    """
    # Instantiate SVC with RBF kernel and class weight balancing.
    estimator = svm.SVC(C=C, gamma=gamma, kernel='rbf', 
                        class_weight='balanced', probability=True)
    # Score the estimator using cross validation.
    cval = cross_val_score(estimator, X_data, y_data, 
                           scoring='f1', cv=2, n_jobs=-1)
    
    return -cval.mean()

In [ ]:
def optimize_svc(X_data, y_data, x0, bounds=None, coolC=10., sigma=0.5, 
                 fs=0.1, burn=10, eps=1e-8, verbose=False):
    """ 
    Simulated Annealing to optimize SVC hyperparameters. 
    
    Parameters
    ----------
    X_data: np.array
        Matrix (2d array) of features.
    y_data: np.array
        Vector (1d array) of targets.
    x0: np.array or list
        Initial values for the variables of the energy function.
        For example, if the energy functions is f(x,y), then
        initial values are given as [x0, y0], where x0 is the 
        initial value for the variable x and y0 for the variable
        y. Order of array elements is important!
    bounds: list of tuples or None, default=None
        List of two-element tuples which define bounds on energy 
        function variables. The number and order of list elements
        is the same as for the array `x0` for the initial values.
        For example, if the energy function is f(x,y), then 
        bounds are defined as follows: [(xl,xu), (yl,yu)], where
        xl, xu are, respectively, lower and upper bounds for the
        variable x, and yl, yu represnt the same limits for the
        variable y. Order of tuples in the list is important!
    coolC: float, default=10.
        Constant decay value of the temperature scheduling.
        Temperature after k-th iteration is determined from 
        the following relation: Tk = T0*exp(-k/C).
    sigma: float, default=0.5
        Standard deviation of a statistical distribution for
        the random walk by which new candidates are generated. 
    fs: float, default=0.1
        Factor for reducing the standard deviation of a statisti-
        cal distribution used for the random walk (see parameter
        `sigma` above for more information). Default value halves
        the `sigma` after the burn-in.    
    burn: int, default=10
        Number of iterations with the original step size of
        the random walk from the Student's t distribution, after
        which the step size is reduced approximately by a factor
        of 10 by switching over to the Normal distribution with
        a lower standard deviation (see parameter `sigma` above).
    eps: float, default=1e-8
        Temperature value at which the algorithm is stopped.
    verbose: bool, default=False
        Indicator for printing (on stdout) internal messages.

    Returns
    -------
    x: np.array
        Optimal values of the SVC hyperparameters "C" and "gamma",
        respectively, as first and second element of this 1d array.
    E: float
        SVC classifier's metric's optimal value.
    """
    
    def svc_crossval(expC, expGamma):
        """ 
        Wrapper for the SVC cross-validation function.

        Parameters
        ----------
        expC: float
            Log value of the SVC regularization parameter. 
        expGamma: float
            Log value of the SVC's RBF kernel coefficient.
        
        Returns
        -------
        cv_score: float
            Negative value of the cross-validated score.
        """
        # Exploring parameters in 'log' space.
        C = 10**expC
        gamma = 10**expGamma
        cv_score = svc_cv(C, gamma, X_data, y_data)
        
        return cv_score

    # Simulated Annealing.
    x, E = simulated_annealing(svc_crossval, x0, bounds=bounds,
                               C=coolC, sigma=sigma, fs=fs, 
                               burn=burn, eps=eps, verbose=verbose)
    
    return x, E

In [ ]:
# Temperature schedule.
x = np.arange(start=1, stop=200, step=1)
T0 = 1.
y1 = T0*np.exp(-x/10)
y2 = T0*0.9**(x)
fig, ax = plt.subplots(1, 2, figsize=(6.5,2.5))
ax[0].plot(x, y1, lw=2, label='T0*exp(-k/10)')
ax[0].plot(x, y2, lw=2, label='T0*0.9**k')
ax[0].set_xlabel('Iterations')
ax[0].set_ylabel('Temperature')
ax[0].legend(loc='upper right')
ax[0].grid()
ax[1].semilogy(x, y1, lw=2, label='T0*exp(-k/10)')
ax[1].semilogy(x, y2, lw=2, label='T0*0.9**k')
ax[1].set_xlabel('Iterations')
ax[1].set_ylabel('Temperature')
ax[1].legend(loc='lower left')
ax[1].grid(which='both')
fig.tight_layout()
plt.show()

In [ ]:
# Initial values (C, gamma) for SVC optimization.
x0 = np.array([1., -2.])

In [ ]:
# Dictionary for holding model scores.
scores_table = {}
# Dictionary for holding SVM parameters.
svm_table = {}

### Principal components analysis (PCA)

In [ ]:
# How many components are needed for the 90% explained variance?
pca = PCA(n_components=0.9, svd_solver='full').fit(X_train)
X_pca = pca.transform(X_test)
print(X_pca.shape)
# Score with the 90% explained variance.
mu, sigma = score_default(X_pca, y_test)

In [ ]:
# Dimensionality reduction.
# Set `whiten` to True/False to see if there is any difference.
pca = PCA(n_components=2, whiten=True).fit(X_train)
X_pca = pca.transform(X_test)

In [ ]:
plot_projection(X_pca, idx_stable)

In [ ]:
mu, sigma = score_default(X_pca, y_test)
scores_table['PCA_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_pca, y_test, x0, bounds=[(-1,3), (-4,1)], 
                    burn=20, eps=1e-10)
print(x, E)
svm_table['PCA'] = x

In [ ]:
mu, sigma = score_optimized(X_pca, y_test, C=10**x[0], gamma=10**x[1])
scores_table['PCA_opt'] = [mu, sigma]

### Dimensionality reduction using KernelPCA

Hyperparameters of the KernelPCA are optimized by means of the **unsupervised** learning.

In [ ]:
def kpca_metric(X_data, gamma=None):
    """
    Distance metric for the kPCA.

    Parameters
    ----------
    X_data: np.array
        Matrix (2d array) of features.
    gamma: float
        Kernel value for the kPCA RBF kernel.
    
    Returns
    -------
    distance: float
        Sum of (euclidean) distances between the original
        features and their reconstruction after embedding
        and its inversion.
    """
    kpca = KernelPCA(n_components=2, kernel='rbf', gamma=gamma,
                     fit_inverse_transform=True, n_jobs=-1)
    X_embedded = kpca.fit_transform(X_data)
    X_reconstructed = kpca.inverse_transform(X_embedded)
    # Compute paired distances between embedding and its reconstruction.
    distances = paired_distances(X_data, X_reconstructed, metric='euclidean')

    return distances.sum()

In [ ]:
def optimize_kpca(X_data, x0, bounds=None, coolC=10., sigma=0.5,
                  fs=0.1, burn=10, eps=1e-8, verbose=False):
    """ Simulated Annealing for kPCA hyperparameters. """

    def kpca(expGamma):
        # Exploring parameters in 'log' space.
        gamma_kpca = 10**expGamma
        distance = kpca_metric(X_data, gamma=gamma_kpca)
        
        return distance

    # Simulated Annealing.
    x, E = simulated_annealing(kpca, x0, bounds=bounds, C=coolC, sigma=sigma, 
                               fs=fs, burn=burn, eps=eps, verbose=verbose)
    
    return x, E

In [ ]:
kpca_metric(X_train)

In [ ]:
# Initial value for the Gamma-kPCA
xk0 = np.array([-2.])
# Optimize kPCA hyperparameters using simulated annealing.
x, E = optimize_kpca(X_train, xk0, bounds=[(-4,1)], burn=20, eps=1e-10)
print(x, E)

In [ ]:
kpca_opt = KernelPCA(n_components=2, kernel='rbf', 
                     gamma=10**x[0], # optimal kernel value
                     n_jobs=-1).fit(X_train)
X_kpca_opt = kpca_opt.transform(X_test)

In [ ]:
plot_projection(X_kpca_opt, idx_stable)

In [ ]:
mu, sigma = score_default(X_kpca_opt, y_test)
scores_table['kPCA_un_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_kpca_opt, y_test, x0, bounds=[(-1,3), (-4,1)], 
                    burn=20, eps=1e-10, verbose=True)
print(x, E)
svm_table['kPCA_un'] = x

In [ ]:
mu, sigma = score_optimized(X_kpca_opt, y_test, C=10**x[0], gamma=10**x[1])
scores_table['kPCA_un_opt'] = [mu, sigma]

Simulated annealing is here used for optimizing the hyperparameters of the KernelPCA and the SVC estimator **at the same time**, using the **supervised** learning.

In [ ]:
def kpca_svc_cv(C, gamma, gamma_kpca, X_data, y_data):
    """ 
    SVC cross-validation with KernelPCA. 
    
    Parameters
    ----------
    C: float
        Regularization parameter of the SVC. 
    gamma: float
        Kernel coefficient of the SVC RBF kernel.
    gamma_kpca; float
        Kernel coefficient of the kPCA RBF kernel.
    X_data: np.array
        Matrix (2d array) of features.
    y_data: np.array
        Vector (1d array) of targets.
    
    Returns
    -------
    cval: float
        Mean value of the score from the cross-validation.
    """
    # Instantiate SVC with RBF kernel and class weight balancing.
    estimator = svm.SVC(C=C, gamma=gamma, kernel='rbf', 
                        class_weight='balanced', probability=True)
    # Instantiate kPCA with RBF kernel.
    reduction = KernelPCA(n_components=2, kernel='rbf', gamma=gamma_kpca)
    # Create a pipeline.
    pipe = Pipeline([
        ('kpca', reduction),
        ('svm', estimator)
    ])
    # Score the pipeline using cross-validation.
    cval = cross_val_score(pipe, X_data, y_data, 
                           scoring='f1', cv=2, n_jobs=-1)
    
    return -cval.mean()

In [ ]:
def optimize_kpca_svc(X_data, y_data, x0, bounds=None,
                      coolC=10., sigma=0.5, fs=0.1, burn=10, 
                      eps=1e-8, verbose=False):
    """ Simulated Annealing for SVC & kPCA hyperparameters. """

    def kpca_svc_crossval(expC, expGamma, expGammkPCA):
        """ 
        Wrapper for the cross-validation function. 
        
        Parameters
        ----------
        expC: float
            Log value of the SVC regularization parameter. 
        expGamma: float
            Log value of the SVC's RBF kernel coefficient.
        expGammkPCA: float
            Log value of the kPCA's RBF kernel coefficient.

        Returns
        -------
        cv_score: float
            Negative value of the cross-validated score.        
        """
        # Exploring parameters in 'log' space.
        C = 10**expC
        gamma = 10**expGamma
        gamma_kpca = 10**expGammkPCA
        cv_score = kpca_svc_cv(C, gamma, gamma_kpca, X_data, y_data)
        
        return cv_score

    # Simulated Annealing.
    x, E = simulated_annealing(kpca_svc_crossval, x0, bounds=bounds,
                               C=coolC, sigma=sigma, fs=fs, burn=burn, 
                               eps=eps, verbose=verbose)
    
    return x, E

In [ ]:
# Initial values (C-SVM, Gamma-SVM, Gamma-kPCA)
xk0 = np.array([1., -2., -1.])
# Optimize kPCA & SVC hyperparameters using simulated annealing.
x, E = optimize_kpca_svc(X_train, y_train, 
                         xk0, bounds=[(-1,3), (-4,1), (-4,1)],
                         burn=20, eps=1e-10)
print(x, E)

In [ ]:
kpca_opt = KernelPCA(n_components=2, kernel='rbf', 
                     gamma=10**x[2], # optimal kernel value
                     n_jobs=-1).fit(X_train)
X_kpca_opt = kpca_opt.transform(X_test)

In [ ]:
plot_projection(X_kpca_opt, idx_stable)

In [ ]:
mu, sigma = score_default(X_kpca_opt, y_test)
scores_table['kPCA_w_def'] = [mu, sigma]

In [ ]:
mu, sigma = score_optimized(X_kpca_opt, y_test, C=10**x[0], gamma=10**x[1])
scores_table['kPCA_w_opt'] = [mu, sigma]

#### KernelPCA without the kPCA kernel optimization

In [ ]:
# Reduce features in the dataset down to only 2 principal components.
kpca = KernelPCA(n_components=2, kernel='rbf', n_jobs=-1).fit(X_train)
X_kpca = kpca.transform(X_test)

In [ ]:
plot_projection(X_kpca, idx_stable)

In [ ]:
mu, sigma = score_default(X_kpca, y_test)
scores_table['kPCA_wo_def'] = [mu, sigma]

In [ ]:
# Apply bounds on SVC hyperparameters in log-space.
# Parameter C: 0.001 to 10000.
# Parameter gamma: 0.0001 to 10.
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_kpca, y_test, 
                    x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10, verbose=True)
print(x, E)
svm_table['kPCA_wo'] = x

In [ ]:
mu, sigma = score_optimized(X_kpca, y_test, C=10**x[0], gamma=10**x[1])
scores_table['kPCA_wo_opt'] = [mu, sigma]

#### Random Search CV for SVC hyperparameters optimization.

In [ ]:
# Search type: ['random', 'halving']
search = 'random'

reduction = KernelPCA(n_components=2, kernel='rbf')
estimator = svm.SVC(kernel='rbf', class_weight='balanced', probability=True)
pipe = Pipeline([
    ('kpca', reduction),
    ('svm', estimator)
])
parameters = {
    'kpca__gamma': stats.expon(scale=.1),
    'svm__C':stats.expon(scale=100), 
    'svm__gamma':stats.expon(scale=.1)
}
if search == 'random':
    model = RandomizedSearchCV(estimator=pipe,
                               param_distributions=parameters,
                               n_iter=200,
                               cv=2, scoring='f1',
                               refit=True, n_jobs=-1)
elif search == 'halving':
    # Experimental method.
    model = HalvingRandomSearchCV(estimator=pipe, 
                                  param_distributions=parameters, 
                                  cv=2, scoring='f1',
                                  refit=True, n_jobs=-1)
else:
    raise NotImplementedError(f'Search method: {search} not recognized.')
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
scores = cross_val_score(model, X_test, y_test, cv=3, scoring='f1', n_jobs=-1)
print('Average score using 3-fold CV: {:.4f} +/- {:.4f}'
      .format(np.mean(scores), np.std(scores)))
scores_table['kPCA_rand'] = [np.mean(scores), np.std(scores)]

### Dimensionality reduction using truncated SVD

In [ ]:
svd = TruncatedSVD(n_components=2).fit(X_train)
X_svd = svd.transform(X_test)

In [ ]:
plot_projection(X_svd, idx_stable)

In [ ]:
mu, sigma = score_default(X_svd, y_test)
scores_table['SVD_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_svd, y_test, x0, bounds=[(-1,3), (-4,1)], 
                    burn=20, eps=1e-10)
print(x, E)
svm_table['SVD'] = x

In [ ]:
mu, sigma = score_optimized(X_svd, y_test, C=10**x[0], gamma=10**x[1])
scores_table['SVD_opt'] = [mu, sigma]

### Isomap embedding

In [ ]:
iso = Isomap(n_components=2, n_neighbors=100, n_jobs=-1).fit(X_train)
X_iso = iso.transform(X_test)

In [ ]:
plot_projection(X_iso, idx_stable)

In [ ]:
mu, sigma = score_default(X_iso, y_test)
scores_table['ISO_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_iso, y_test, x0, bounds=[(-1,3), (-4,1)], 
                    burn=20, eps=1e-10)
print(x, E)
svm_table['ISO'] = x

In [ ]:
mu, sigma = score_optimized(X_iso, y_test, C=10**x[0], gamma=10**x[1])
scores_table['ISO_opt'] = [mu, sigma]

### t-distributed Stochastic Neighbor Embedding (t-SNE)

#### t-SNE with optimized hyperparameters

Here hyperparameters of the t-SNE (i.e. `perplexity`) is optimized.

In [ ]:
def tsne_metric(perplex, X_data):
    """ t-SNE metric based on the KL divergence. """
    from numpy import log

    reduction = TSNE(n_components=2, perplexity=perplex, n_jobs=-1)
    # Score the estimator.
    cval = reduction.fit(X_data)

    kl = cval.kl_divergence_
    n = float(X_data.shape[0])
    parameters = reduction.get_params(deep=False)
    perp = parameters['perplexity']

    metric = 2*kl + log(n)*(perp/n)
    
    return metric

In [ ]:
def optimize_tsne(X_data, x0, bounds=None, coolC=10., sigma=0.5, 
                  fs=0.1, burn=10, eps=1e-6, verbose=False):
    """ Simulated Annealing for t-SNE hyperparameters. """

    def tsne(expPerplex):
        from numpy import exp

        # Exploring parameter perplexity in natural logarithm (ln) space.
        perplex = exp(expPerplex)
        # KL divergence based score.
        kl_metric = tsne_metric(perplex, X_data)
        
        return kl_metric

    # Simulated Annealing.
    x, E = simulated_annealing(tsne, x0, bounds=bounds, C=coolC, 
                               sigma=sigma, fs=fs, burn=burn, 
                               eps=eps, verbose=verbose)
    
    return x, E

In [ ]:
# Initial values (perplexity)
xt0 = np.array([3.])
# Optimize t-SNE & SVC hyperparameters using simulated annealing.
x, E = optimize_tsne(X_train, xt0, bounds=[(0.,4.)], sigma=0.2,
                     burn=20, eps=1e-8, verbose=True)
print(x, E)

In [ ]:
X_tsne_opt = TSNE(n_components=2, 
                  perplexity=np.exp(x[0]), # optimal value
                  n_jobs=-1).fit_transform(X_test)

In [ ]:
plot_projection(X_tsne_opt, idx_stable)

In [ ]:
mu, sigma = score_default(X_tsne_opt, y_test)
scores_table['SNE_w_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_tsne_opt, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['SNE_w'] = x

In [ ]:
mu, sigma = score_optimized(X_tsne_opt, y_test, C=10**x[0], gamma=10**x[1])
scores_table['SNE_w_opt'] = [mu, sigma]

#### t-SNE without hyperparameters optimization

Hyperparameters of the t-SNE are not being optimized.

In [ ]:
X_tsne = TSNE(n_components=2, n_jobs=-1).fit_transform(X_test)

In [ ]:
plot_projection(X_tsne, idx_stable)

In [ ]:
mu, sigma = score_default(X_tsne, y_test)
scores_table['SNE_wo_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_tsne, y_test, x0, bounds=[(-1,3), (-4,1)], 
                    burn=20, eps=1e-10)
print(x, E)
svm_table['SNE_wo'] = x

In [ ]:
mu, sigma = score_optimized(X_tsne, y_test, C=10**x[0], gamma=10**x[1])
scores_table['SNE_wo_opt'] = [mu, sigma]

### Locally Linear Embedding (LLE)

In [ ]:
lle = LLE(n_components=2, n_neighbors=10, 
          method='standard', n_jobs=-1).fit(X_train)
X_lle = lle.transform(X_test)

In [ ]:
plot_projection(X_lle, idx_stable)

In [ ]:
mu, sigma = score_default(X_lle, y_test)
scores_table['LLE_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_lle, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['LLE'] = x

In [ ]:
mu, sigma = score_optimized(X_lle, y_test, C=10**x[0], gamma=10**x[1])
scores_table['LLE_opt'] = [mu, sigma]

### Locally linear embedding with LTSA

In [ ]:
ltsa = LLE(n_components=2, n_neighbors=10, 
           method='ltsa', eigen_solver='dense', n_jobs=-1).fit(X_train)
X_ltsa = ltsa.transform(X_test)

In [ ]:
plot_projection(X_ltsa, idx_stable)

In [ ]:
mu, sigma = score_default(X_ltsa, y_test)
scores_table['TSA_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_ltsa, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['TSA'] = x

In [ ]:
mu, sigma = score_optimized(X_ltsa, y_test, C=10**x[0], gamma=10**x[1])
scores_table['TSA_opt'] = [mu, sigma]

### Locally linear embedding with Hessian

In [ ]:
hess = LLE(n_components=2, n_neighbors=100, 
           method='hessian', n_jobs=-1).fit(X_train)
X_hess = hess.transform(X_test)

In [ ]:
plot_projection(X_hess, idx_stable)

In [ ]:
mu, sigma = score_default(X_hess, y_test)
scores_table['HES_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_hess, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['HES'] = x

In [ ]:
mu, sigma = score_optimized(X_hess, y_test, C=10**x[0], gamma=10**x[1])
scores_table['HES_opt'] = [mu, sigma]

### Modified locally linear embedding (MLLE)

In [ ]:
mlle = LLE(n_components=2, n_neighbors=50, 
           method='modified', n_jobs=-1).fit(X_train)
X_mlle = lle.transform(X_test)

In [ ]:
plot_projection(X_mlle, idx_stable)

In [ ]:
mu, sigma = score_default(X_mlle, y_test)
scores_table['MLE_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_mlle, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['MLE'] = x

In [ ]:
mu, sigma = score_optimized(X_mlle, y_test, C=10**x[0], gamma=10**x[1])
scores_table['MLE_opt'] = [mu, sigma]

### Spectral embedding

In [ ]:
X_spec = SE(n_components=2, affinity='nearest_neighbors', 
            n_jobs=-1).fit_transform(X_test)

In [ ]:
plot_projection(X_spec, idx_stable)

In [ ]:
mu, sigma = score_default(X_spec, y_test)
scores_table['SPE_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_spec, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['SPE'] = x

In [ ]:
mu, sigma = score_optimized(X_spec, y_test, C=10**x[0], gamma=10**x[1])
scores_table['SPE_opt'] = [mu, sigma]

### Multi-dimensional scaling (MDS)

In [ ]:
# Set `metric` to True/False to see if there is any difference.
X_mds = MDS(n_components=2, metric=True, n_jobs=-1).fit_transform(X_test)

In [ ]:
plot_projection(X_mds, idx_stable)

In [ ]:
mu, sigma = score_default(X_mds, y_test)
scores_table['MDS_def'] = [mu, sigma]

In [ ]:
# Optimize SVC hyperparameters using simulated annealing.
x, E = optimize_svc(X_mds, y_test, x0, bounds=[(-1,3), (-4,1)],
                    burn=20, eps=1e-10)
print(x, E)
svm_table['MDS'] = x

In [ ]:
mu, sigma = score_optimized(X_mds, y_test, C=10**x[0], gamma=10**x[1])
scores_table['MDS_opt'] = [mu, sigma]

In [ ]:
# Aggregate scores from all models into a dataframe.
pdscores = pd.DataFrame(data=scores_table)
pdscores = pdscores.transpose()
pdscores.columns = ['Mean', 'Std']
pdscores

**Note**: Interesting models for additional testing and comparisons: 'kPCA', 'SVD', 'ISO', 'MDS' and 't-SNE'.

In [ ]:
# Aggregate hyperparameters from different SVMs into a dataframe.
svms = pd.DataFrame(data=svm_table)
svms = svms.transpose()
svms.columns = ['C', 'Gamma']
# Convert back from the log-scale.
svms = svms.apply(lambda x: 10**x)
svms

### Precision-Recall Tradeoff for a selected model

In [ ]:
# Select SVC model here from the following list:
# ['PCA', 'kPCA_un', 'kPCA_wo', 'SVD', 'ISO', 'SNE_w', 'SNE_wo', 'LLE',
#  'TSA', 'HES', 'MLE', 'SPE', 'MDS']
model = 'kPCA_wo'
C = svms['C'].loc[model]
gamma = svms['Gamma'].loc[model]
best_parameters = {'C': C, 'gamma': gamma}

In [ ]:
y_probas = cross_val_predict(svm.SVC(**best_parameters, probability=True, 
                                     class_weight='balanced'), 
                             X_test, y_test, cv=3, 
                             method='predict_proba',
                             n_jobs=-1)

In [ ]:
y_scores = y_probas[:,1]  # score == probability of positive class
precisions, recalls, thresholds = metrics.precision_recall_curve(y_test, y_scores)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(precisions, recalls, lw=2, label='SVC')
default = np.argmin(np.abs(thresholds - 0.5))
ax.plot(precisions[default], recalls[default], '^', c='k', markersize=8, 
        label='Threshold = 0.5', fillstyle='none', mew=2)
ax.set_xlabel('Precision')
ax.set_ylabel('Recall')
ax.legend(loc='best')
ax.grid()
fig.tight_layout()
plt.show()

#### Plot decision region for the selected model

In [ ]:
# Projected data (X_test) that is consistent 
# with the previously selected model.
if model == 'PCA':
    X_test_mod = X_pca
elif model == 'kPCA_un':
    X_test_mod = X_kpca_opt
elif model == 'kPCA_wo':
    X_test_mod = X_kpca
elif model == 'SVD':
    X_test_mod = X_svd
elif model == 'ISO':
    X_test_mod = X_iso
elif model == 'SNE_w':
    X_test_mod = X_tsne_opt
elif model == 'SNE_wo':
    X_test_mod = X_tsne
elif model == 'LLE':
    X_test_mod = X_lle
elif model == 'TSA':
    X_test_mod = X_ltsa
elif model == 'HES':
    X_test_mod = X_hess
elif model == 'MLE':
    X_test_mod = X_mlle
elif model == 'SPE':
    X_test_mod = X_spec
elif model == 'MDS':
    X_test_mod = X_mds
else:
    raise NotImplementedError(f'Model designation: {model} not recognized!')

In [ ]:
# Generate SVC from the selected projection and
# associated optimized model parameters.
svc_best = svm.SVC(C=C, gamma=gamma, kernel='rbf', 
                   class_weight='balanced', 
                   probability=True).fit(X_test_mod, y_test)

In [ ]:
h = 0.1; delta = 0.1
x_min, x_max = X_test_mod[:,0].min() - h, X_test_mod[:,0].max() + h
y_min, y_max = X_test_mod[:,1].min() - h, X_test_mod[:,1].max() + h
xx, yy = np.meshgrid(np.arange(x_min, x_max, delta), np.arange(y_min, y_max, delta))
Z = svc_best.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
Z = Z.reshape(xx.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
ax.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu, alpha=0.8)
ax.scatter(X_test_mod[idx_stable,0], X_test_mod[idx_stable,1], 
           s=30, c='green', marker='o', edgecolors='k', alpha=0.5, label='Stable')
ax.scatter(X_test_mod[~idx_stable,0], X_test_mod[~idx_stable,1], 
           s=30, c='red', marker='o', edgecolors='k', alpha=0.5, label='Unstable')
ax.legend(loc='upper left')
ax.set_xlabel('1st component')
ax.set_ylabel('2nd component')
# Re-set limits if needed.
#ax.set_xlim(right=70)
#ax.set_ylim(top=70)
ax.grid()
plt.show()

#### Computing environment

In [ ]:
import sys, IPython, sklearn, scipy, matplotlib, pandas, numpy
print("Notebook createad with:\
      \nPython {:s}\nIPython {:s}\nScikit-learn {:s}\nPandas {:s}\nNumpy \
      {:s}\nScipy {:s}\nMatplotlib {:s}"\
      .format(sys.version[:5], IPython.__version__, sklearn.__version__, 
              pandas.__version__, numpy.__version__, scipy.__version__, 
              matplotlib.__version__))